In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
# model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

In [2]:
# for k,v in sd.items():
    # print(k,v.shape)

nn.ModuleDict is used to store a collection of PyTorch modules (layers or networks) as key-value pairs, similar to a regular Python dictionary where keys are strings and values are PyTorch modules.



ChatGPT
In Python, @dataclass is a decorator introduced in Python 3.7 as part of the dataclasses module. It provides a way to automatically generate special methods such as __init__, __repr__, __eq__, __ne__, __lt__, __le__, __gt__, and __ge__ for classes. These methods are typically needed when you want to create a class that primarily serves as a data container (like a struct in C or a plain data class in other languages).



the method tensor.contiguous() is used to create a copy of a tensor with the same data but stored in a contiguous block of memory

Sometimes, due to operations like slicing, transposing, or reshaping, tensors may become non-contiguous. This means their elements are no longer stored in a single, contiguous block of memory. Non-contiguous tensors may have their elements scattered across memory, requiring extra bookkeeping to access elements in the correct order.

In [2]:
import torch
from dataclasses import dataclass
import torch.nn as nn
from torch.nn import functional as F
import math
#####
class CausalAttention(nn.Module):
  def __init__(self, config):
    super().__init__()
    assert config.n_embd % config.n_head == 0
    # key, query, value projections for all heads, but in a batch
    self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
    # output projection
    self.c_proj = nn.Linear(config.n_embd, config.n_embd)
    self.c_proj.NANOGPT_SCALE_INIT=1
    #regularization
    self.n_head = config.n_head
    self.n_embd = config.n_embd
    # causal mask to ensure that attention is only applied to the left in the input sequence
    self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size)).view(1, 1, config.block_size, config.block_size))
    #tensor([[[[1., 0., 0., 0.],
             # [1., 1., 0., 0.],
             # [1., 1., 1., 0.],
             # [1., 1., 1., 1.]]]])
    #self.register_buffer(name, tensor) is a method used to register a tensor as a buffer non trainable parameter of a nn.Module subclass
  def forward(self, x):
    B, T, C = x.size()  #batch size, seq len , token embedding size
    # calculate query, key, values for all heads in batch and move head forward to be the batch dim
    qkv = self.c_attn(x)
    q,k,v = qkv.split(self.n_embd,dim=2)
    q = q.view(B,T,self.n_head,C//self.n_head).transpose(1,2) # B,nh, T,hs
    k = k.view(B,T,self.n_head,C//self.n_head).transpose(1,2) # B,nh, T,C/nh
    v = v.view(B,T,self.n_head,C//self.n_head).transpose(1,2) # B,nh, T,C/nh
    # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
    att = (q @ k.transpose(-2,-1)) * (1.0 / math.sqrt(k.size(-1)))  # after transpose (B,nh,hs,T)
    att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))#fill all place of attn where self.mask ==0
    att = F.softmax(att, dim=-1)
    y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
    y = y.transpose(1,2).contiguous().view(B,T,C) # re-assemble all head outputs side by side
    y = self.c_proj(y)
    return y

class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT=1
    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x


class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x



@dataclass
class GPTConfig:
    block_size: int = 1024
    vocab_size: int = 50257
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768

class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        # weight teing of embedding and last layer weight. both use same weight.
        # we want similarity between vector of similar word and similarly we want almost same probab. for similar words
        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.transformer.wte.weight = self.lm_head.weight

        #init weight
        #self.apply(fn) is a method used in nn.Module subclasses to apply a function fn to every submodule (layer) within that module.
        self.apply(self._init_weights)

    def _init_weights(self, module):
        std=.02
        if isinstance(module, nn.Linear):
          if hasattr(module, 'NANOGPT_SCALE_INIT'):
            std *= (2*self.config.n_layer)**-0.5
          torch.nn.init.normal_(module.weight, mean=0.0, std=std)
          if module.bias is not None:
            torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
          torch.nn.init.normal_(module.weight, mean=0.0, std=std)


    def forward(self, idx, target=None):
      B, T = idx.size()
      assert T < self.config.block_size, f"Cannot forward sequence of length {T}, block size is only {self.config.block_size}"
      pos = torch.arange(0, T, dtype=torch.long, device=idx.device)
      tok_emb = self.transformer.wte(idx)
      pos_emb = self.transformer.wpe(pos)
      x = tok_emb + pos_emb
      for block in self.transformer.h:
        x = block(x)
      x = self.transformer.ln_f(x)
      logits = self.lm_head(x) #(B, T, vocab_size)
      loss = None
      if target is not None:
        logits = logits.view(-1, logits.size(-1)) # B*T , vocab_size
        target = target.view(-1)         # B*T
        loss = F.cross_entropy(logits, target) #
      return logits,loss

    @classmethod
    def from_pretrained(cls, model_type):
        """Loads pretrained GPT-2 model weights from huggingface"""
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

In [4]:
%%capture
pip install tiktoken

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
num = 4
max_length = 30
# model = GPT.from_pretrained("gpt2")
model = GPT(GPTConfig())
model.eval()
model.to(device)
print("ok")

ok


In [5]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-06-16 05:55:16--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-06-16 05:55:16 (23.8 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [6]:
import tiktoken
class DataLoaderLite:
  def __init__(self, B, T):
    self.B = B
    self.T = T
    with open('input.txt', 'r', encoding='utf-8') as f:
        text = f.read()
    enc = tiktoken.get_encoding("gpt2")
    tokens = enc.encode(text)
    print(f"unique tokens {len(set(tokens))}")
    self.tokens = torch.tensor(tokens)
    print(f"loaded {len(self.tokens)} tokens")
    print(f"1 epoch  = {len(self.tokens) // (B*T)} batches")

    self.current_position = 0

  def nex_batch(self):
    B, T = self.B ,self.T
    buf = self.tokens[self.current_position:self.current_position+B*T+1]

    x = buf[:-1].view(B,T)
    y = buf[1:].view(B,T)

    self.current_position += B*T
    if self.current_position + B*T+1 >= len(self.tokens):
      self.current_position = 0
    return x,y


ModuleNotFoundError: No module named 'tiktoken'

In [ ]:
torch.manual_seed(1337)
if torch.cuda.is_available():
  torch.cuda.manual_seed(1337)
train_loader = DataLoaderLite(num, max_length)

In [10]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

import time
for i in range(50):
  t0 = time.time()
  x,y = train_loader.nex_batch()
  x = x.to(device)
  y = y.to(device)
  optimizer.zero_grad()
  logits, loss = model(x,y)
  # import code; code.interact(local=locals())
  loss.backward()
  optimizer.step()
  torch.cuda.synchronize()
  t1 = time.time()
  print(f"step{i} - {loss.item()} time {t1-t0}")
  # print(f"step{i} - {loss.item()}")

step0 - 10.960549354553223
step1 - 9.74317741394043
step2 - 9.282991409301758
step3 - 9.148523330688477
step4 - 8.465890884399414
step5 - 8.713939666748047
step6 - 8.766549110412598
step7 - 8.407317161560059
step8 - 8.635539054870605
step9 - 8.242499351501465
step10 - 7.825723171234131
step11 - 8.08593463897705
step12 - 7.452076435089111
step13 - 7.500515460968018
step14 - 7.2950615882873535
step15 - 7.422034740447998
step16 - 7.208282947540283
step17 - 7.422486305236816
step18 - 7.997808456420898
step19 - 7.450544357299805
step20 - 7.721587657928467
step21 - 7.4293107986450195
step22 - 7.499777793884277
step23 - 6.916680812835693
step24 - 6.9305572509765625
step25 - 6.6781463623046875
step26 - 6.612472057342529
step27 - 6.800876140594482
step28 - 6.742293357849121
step29 - 7.93330717086792
step30 - 6.77671480178833
step31 - 6.972308158874512
step32 - 7.033783435821533
step33 - 6.958427906036377
step34 - 7.001629829406738
step35 - 6.8131208419799805
step36 - 8.075823783874512
step37 - 

In [11]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")
tokens = enc.encode("hello i am langauge model")
tokens = torch.tensor(tokens , dtype=torch.long)

tokens = tokens.unsqueeze(0).repeat(num,1)

tokens.shape
x = tokens.to(device)

while(x.shape[1] < max_length):
  with torch.no_grad():
    logits,loss = model(x)
    logits = logits[:,-1,:]
    probs = F.softmax(logits, dim=-1)
    topk_prob, topk_index = torch.topk(probs, k=50, dim=-1)
    y = torch.multinomial(topk_prob, num_samples=1)
    xx = torch.gather(topk_index, -1, y)
    x = torch.cat((x,xx),dim=1)

for i in range(num):
  print(enc.decode(x[i].cpu().numpy()))
  print("--------")

hello i am langauge model,

 and
, my is-, I:
 areIA of:N would-, him:
--------
hello i am langauge model my,
:
.

ius '.


. it,,
s fromMAR,
--------
hello i am langauge model: IN;'llius.

 I; us, know.
 a he it: I his;
--------
hello i am langauge model:
.

:




,

 shall: and the:
; him me
--------


In [12]:
# GPT.from_pretrained("gpt2")

